In [7]:
import xmltodict
import math
import urllib.request
import itertools
import pandas as pd
import pickle   
from sklearn.utils import shuffle
import time
import numpy as np
import traceback

In [8]:
import sqlite3
from sqlite3 import Error

def create_con(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
        conn.commit()
    except Error as e:
        print(e)

def create_game(conn, game):
    sql = """ INSERT INTO games(name)
              VALUES(?) """
    c = conn.cursor()
    c.execute(sql, game)
    conn.commit()
    return c.lastrowid


def create_reviews(conn, reviews):
    sql = """ INSERT INTO reviews(user, rating, comment, ID)
              VALUES(?,?,?,?) """
    cur = conn.cursor()
    for review in reviews:
        cur.execute(sql,review)
    cur.execute(sql, review)
    conn.commit()
    return cur.lastrowid

def do_sql(conn,sql):
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)

def count_reviews(conn):
    cur = conn.cursor()
    print(cur.execute("SELECT COUNT (*) FROM reviews;").fetchall()[0])

def count_distinct_reviews(conn):
    cur = conn.cursor()
    print(cur.execute("SELECT COUNT(*) FROM (SELECT DISTINCT * FROM reviews);").fetchall()[0])

def del_all_records(conn):
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS reviews")
    cur.execute("DROP TABLE IF EXISTS games")
    conn.commit()

In [9]:
from urllib.request import urlopen
# the original csv from https://raw.githubusercontent.com/beefsack/bgg-ranking-historicals/master/2017-02-21.csv
# The ID's where used in API calls to retrieve the game reviews
link="https://raw.githubusercontent.com/beefsack/bgg-ranking-historicals/master/2020-08-19.csv"
f = urlopen(link)
games = pd.read_csv(f)
games.describe()
games.sort_values('Users rated',ascending=False,inplace=True)
games[:5]

,ID,Name,Year,Rank,Average,Bayes average,Users rated,URL,Thumbnail
90,30549,Pandemic,2008,91,7.62,7.518,96186,/boardgame/30549/pandemic,https://cf.geekdo-images.com/micro/img/0m3-oqB...
172,822,Carcassonne,2000,173,7.42,7.311,96181,/boardgame/822/carcassonne,https://cf.geekdo-images.com/micro/img/z0tTaij...
380,13,Catan,1995,381,7.16,7.001,96171,/boardgame/13/catan,https://cf.geekdo-images.com/micro/img/e0y6Bog...
49,68448,7 Wonders,2010,50,7.76,7.662,79830,/boardgame/68448/7-wonders,https://cf.geekdo-images.com/micro/img/h-Ejv31...
87,36218,Dominion,2008,88,7.63,7.521,74933,/boardgame/36218/dominion,https://cf.geekdo-images.com/micro/img/VYp2s2f...


In [7]:
from sqlalchemy import create_engine

sql_create_table_reviews = """ CREATE TABLE IF NOT EXISTS reviews(
                                    review_id integer PRIMARY KEY,
                                    user text NOT NULL,
                                    rating NOT NULL,
                                    comment text,              
                                    ID integer,
                                    FOREIGN KEY(ID) REFERENCES games(ID));"""

conn = create_con('test.db')
del_all_records(conn) # empty database (optional)

if conn:
    engine = create_engine('sqlite:///%s' % 'test.db', echo=True)
    games[['ID','Name']].to_sql('games', con=engine)
    # c = conn.cursor()
    # c.execute(""" DROP TABLE IF EXISTS games""")
    # create_table(conn, sql_create_table_games)
    # c.execute(""" DROP TABLE IF EXISTS reviews""")
    create_table(conn, sql_create_table_reviews)
    count_reviews(conn)


2020-08-19 21:00:05,058 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-08-19 21:00:05,066 INFO sqlalchemy.engine.base.Engine ()
2020-08-19 21:00:05,073 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-08-19 21:00:05,080 INFO sqlalchemy.engine.base.Engine ()
2020-08-19 21:00:05,090 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("games")
2020-08-19 21:00:05,100 INFO sqlalchemy.engine.base.Engine ()
2020-08-19 21:00:05,108 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("games")
2020-08-19 21:00:05,119 INFO sqlalchemy.engine.base.Engine ()
2020-08-19 21:00:05,137 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE games (
	"index" BIGINT, 
	"ID" BIGINT, 
	"Name" TEXT
)


2020-08-19 21:00:05,142 INFO sqlalchemy.engine.base.Engine ()
2020-08-19 21:00:05,170 INFO sqlalchemy.engine.base.Engine COMMIT
2020-08-19 21:00:05,173 INFO sqlalchemy.engine.base.Engine CREATE INDEX i

In [8]:
def gen_batches():
    num_batches = len(games)//100
    for b in range(num_batches):
        result = slice(b*100,(b+1)*100)
        if b*100>-1:
            yield result
    yield slice((b+1)*100,len(games)+1)

In [9]:
def number_rating_pages(game_id):
    url = 'https://www.boardgamegeek.com/xmlapi2/thing?id='+str(game_id)+'&ratingcomments=1'
    u = urllib.request.urlopen(url).read()#The url you want to open
    doc = xmltodict.parse(u)
    return math.ceil(int(doc['items']['item']['comments']['@totalitems'])/100)


In [10]:
def get_url_result(ids,p): # given a url of an object and a page, return list of dicts with the comments
    url = 'https://www.boardgamegeek.com/xmlapi2/thing?id='+ids+'&ratingcomments=1'
    url += '&&page='+str(p)
    u = urllib.request.urlopen(url).read()#The url you want to open
    doc = xmltodict.parse(u)
    return doc 



In [11]:
def process_result(url_result):
    remove_list = []
    for g in url_result['items']['item']:
        if not isinstance(g, str):
            if 'comment' in g['comments'].keys():
                try:
                    review_tup = [(g['@id'],) + tuple(review.values()) for review in g['comments']['comment']]
                    create_reviews(conn,review_tup)
                except Exception as e:
                    print(e,traceback.format_exc())
            else:
                remove_list.append(int(g['@id']))
        else:
            print('crazy str',g)
    return remove_list

In [14]:
conn = create_con('test.db')
game_generator = gen_batches()

# get all games, but do this per batch of 100 games
for batch in game_generator: 
    print('new batch,', batch)
    # list of games
    games_in_batch = list(games['ID'][batch])
    
    #get maximum number of pages from game on top of the batch
    num_pages = number_rating_pages(games_in_batch[0])+1
    if num_pages>0:
        # get all the pages, quering as little games as possible and store the results
        for p in range(0,num_pages):
            for tryout in range(5):
                try:
                    print('next query for page {} for {} games'.format(p,len(games_in_batch)))
                    #print('games still in batch are \n',games_in_batch)
                    if not games_in_batch:
                        print('gamelist empty!')
                        break
                    # convert list of current games to url input
                    ids = ','.join(str(g) for g in games_in_batch)
                    #query API and parse to dictionary
                    url_result = get_url_result(ids,p)
                    print('data retrieved')
                    # remove games that have no comment, they dont have to be queried again
                    removelist = process_result(url_result)
                    #print('to remove',removelist)
                    games_in_batch = [g for g in games_in_batch if g not in removelist]
                    time.sleep(8)
                    # if p%1000==0:
                    #     with open('ratings.pickle', 'wb') as handle:
                    #         pickle.dump(rating_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
                    break
                except Exception as e:
                    print('error',tryout,p,e)
                    time.sleep(30)
print('all done')
conn.close()

new batch, slice(0, 100, None)
next query for page 0 for 100 games
data retrieved
next query for page 1 for 100 games
data retrieved
next query for page 2 for 100 games
data retrieved
next query for page 3 for 100 games
data retrieved
next query for page 4 for 100 games
data retrieved
next query for page 5 for 100 games
data retrieved
next query for page 6 for 100 games
data retrieved
next query for page 7 for 100 games
data retrieved
next query for page 8 for 100 games
data retrieved
next query for page 9 for 100 games
data retrieved
next query for page 10 for 100 games
data retrieved
next query for page 11 for 100 games
data retrieved
next query for page 12 for 100 games
data retrieved
next query for page 13 for 100 games
data retrieved
next query for page 14 for 100 games
data retrieved
next query for page 15 for 100 games
data retrieved
next query for page 16 for 100 games
data retrieved
next query for page 17 for 100 games
data retrieved
next query for page 18 for 100 games
data r

In [15]:
# get tables from db
con = sqlite3.connect('test.db')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('games',), ('reviews',)]


In [21]:
for val in ['user','rating','ID']:
    print(val,cur.execute("SELECT COUNT(*) FROM (SELECT DISTINCT "+ val +" FROM reviews);").fetchall())

user [(19330,)]
rating [(351049,)]
ID [(2743476,)]


# Import DB

In [1]:
import pandas as pd
import sqlite3

# Create your connection.
cnx = sqlite3.connect('test.db')
from aoc import timeit
with timeit(): df = pd.read_sql_query("SELECT * FROM reviews", cnx)

INFO:root:None, time: 7.077454209327698


In [4]:
df.to_csv('17mreviews.csv')

INFO:root:None, time: 1.8687207261721293


351048

In [4]:
df.rename(columns={'user':'ID','rating':'user','comment':'rating','ID':'comment'},inplace=True)

In [5]:
df = df[['user', 'rating', 'comment','ID']]
df['ID'] = df['ID'].astype('int64')

In [17]:
df = df.merge(games[['ID','Name']],on='ID',how='left')

In [18]:
df.head()

,user,rating,comment,ID,Name
0,Torsten,10.0,NaN,30549,Pandemic
1,mitnachtKAUBO-I,10.0,Hands down my favorite new game of BGG CON 200...,30549,Pandemic
2,avlawn,10.0,I tend to either love or easily tire of co-op ...,30549,Pandemic
3,Mike Mayer,10.0,NaN,30549,Pandemic
4,Mease19,10.0,This is an amazing co-op game. I play mostly ...,30549,Pandemic


In [19]:
len(df)

17679583

In [20]:
df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)

In [21]:
len(df)

15864308

In [22]:
df.drop_duplicates(subset=['user','ID'], keep='first', inplace=True, ignore_index=True)

In [23]:
len(df)

15823269

In [24]:
df['user'].nunique()

1.002593585434211

In [26]:
filename = 'bgg-20m-reviews.csv'
df.rename(columns={'Name':'name'},inplace=True)
df.to_csv(filename)


In [30]:
import pandas as pd
import sqlite3
import aoc
filename = 'bgg-20m-reviews.csv'
with aoc.timeit(): df = pd.read_csv(filename,index_col=0)